In [1]:
import math

def calculate_training_params(model_params, seq_length=1024, batch_size=256, warmup_fraction=0.01):
    """
    Calculate optimal training parameters for a given model size based on Chinchilla scaling laws.
    
    Args:
        model_params (int): Number of model parameters in billions (e.g., 1 for 1B).
        seq_length (int): Sequence length used in training (default is 1024).
        batch_size (int): Batch size for training (default is 256).
        warmup_fraction (float): Fraction of total training steps used for warmup (default is 0.01).
    
    Returns:
        dict: Dictionary containing training parameters.
    """
    # Constants
    tokens_per_param = 20  # Chinchilla scaling: 20 tokens per parameter
    optimal_tokens = model_params * tokens_per_param * 1e9  # Total tokens to train on
    
    # Calculate total training steps
    steps_per_epoch = optimal_tokens / (batch_size * seq_length)
    total_training_steps = math.ceil(steps_per_epoch)
    
    # Warmup steps
    warmup_steps = math.ceil(warmup_fraction * total_training_steps)
    
    # Optimizer hyperparameters (based on practical defaults)
    if model_params < 1:  # For smaller models (<1B)
        lr = 2e-4
        weight_decay = 0.1
    elif model_params <= 10:  # For medium-sized models (1B-10B)
        lr = 3e-4
        weight_decay = 0.1
    else:  # For larger models (>10B)
        lr = 1e-4
        weight_decay = 0.01
    
    # Expected final loss (heuristic based on model size)
    expected_loss = 1.69 - 0.03 * math.log10(model_params) 
    # update the expected_loss with Chinchilla scaling law
    # expected_loss = 1.69 + 406.4/(model_params**0.28) + 410/(model_params**0.38)
    
    return {
        "Optimal Training Tokens": f"{optimal_tokens:.2e} tokens",
        "Batch Size": batch_size,
        "Sequence Length": seq_length,
        "Total Training Steps": total_training_steps,
        "Warmup Steps": warmup_steps,
        "Learning Rate (LR)": lr,
        "Weight Decay": weight_decay,
        "Expected Final Loss": round(expected_loss, 4)
    }


In [2]:
print("Welcome to the LLM Training Parameter Calculator!")
# model_params = float(input("Enter the model parameters (in billions, e.g., 1 for 1B): "))
# seq_length = int(input("Enter the sequence length (default: 1024): ") or 1024)
# batch_size = int(input("Enter the batch size (default: 256): ") or 256)
model_params = 0.145
seq_length = 1024
batch_size = 64

training_params = calculate_training_params(model_params, seq_length, batch_size)
print("\n=== Training Parameters ===")
for key, value in training_params.items():
    print(f"{key}: {value}")


Welcome to the LLM Training Parameter Calculator!

=== Training Parameters ===
Optimal Training Tokens: 2.90e+09 tokens
Batch Size: 64
Sequence Length: 1024
Total Training Steps: 44251
Warmup Steps: 443
Learning Rate (LR): 0.0002
Weight Decay: 0.1
Expected Final Loss: 1.7152


In [3]:
# GAN Loss
import torch
import torch.nn as nn

# Set up device for computation
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

# Define Binary Cross-Entropy Loss for Vanilla GANs
bce_loss = nn.BCELoss()

# Define a sample batch size and the size of latent space
batch_size = 4
latent_dim = 100

# Fake and real labels for the discriminator
real_labels = torch.ones(batch_size, 1).to(device)  # Real labels (1s)
fake_labels = torch.zeros(batch_size, 1).to(device)  # Fake labels (0s)

# Inputs for Discriminator Loss
# Real data (from the dataset)
real_data = torch.randn(batch_size, 3, 64, 64).to(device)  # Simulated real images (e.g., 3 channels, 64x64)

# Fake data (generated by the Generator)
generator = nn.Sequential(
    nn.Linear(latent_dim, 3 * 64 * 64),  # Fully connected layer to project latent space to image space
    nn.Tanh()  # Tanh activation to output images with values between -1 and 1
).to(device)

latent_vectors = torch.randn(batch_size, latent_dim).to(device)  # Random noise (latent space)
fake_data = generator(latent_vectors).view(batch_size, 3, 64, 64)  # Reshape to image dimensions

# Discriminator (a simple model for this demonstration)
discriminator = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3 * 64 * 64, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid()  # Outputs probabilities between 0 and 1
).to(device)

# Forward pass for real and fake data through the discriminator
real_output = discriminator(real_data)  # Discriminator output for real data
fake_output = discriminator(fake_data.detach())  # Discriminator output for fake data

# Calculate Discriminator Loss using Binary Cross-Entropy
# Loss for real data (discriminator should classify it as real)
loss_real = bce_loss(real_output, real_labels)

# Loss for fake data (discriminator should classify it as fake)
loss_fake = bce_loss(fake_output, fake_labels)

d_loss = loss_real + loss_fake  # Total discriminator loss

# Generator Loss (goal: fool the discriminator into classifying fake data as real)
g_loss = bce_loss(discriminator(fake_data), real_labels)  # Use real_labels to simulate "fooling"

# Print outputs for Binary Cross-Entropy Loss
print("Binary Cross-Entropy Loss:")
print("Discriminator output (real):", real_output)
print("Discriminator output (fake):", fake_output)
print("Discriminator loss (real):", loss_real.item())
print("Discriminator loss (fake):", loss_fake.item())
print("Total Discriminator Loss:", d_loss.item())
print("Generator Loss:", g_loss.item())

# Least Squares Loss for LSGAN
# Define real and fake labels for Least Squares Loss
real_labels_ls = torch.ones(batch_size, 1).to(device)  # Real labels (target = 1)
fake_labels_ls = torch.zeros(batch_size, 1).to(device)  # Fake labels (target = 0)

# Calculate Discriminator Loss using Least Squares Loss
loss_real_ls = 0.5 * torch.mean((real_output - real_labels_ls) ** 2)  # Loss for real data
loss_fake_ls = 0.5 * torch.mean((fake_output - fake_labels_ls) ** 2)  # Loss for fake data

d_loss_ls = loss_real_ls + loss_fake_ls  # Total discriminator loss

# Generator Loss (goal: fool the discriminator into classifying fake data as real)
g_loss_ls = 0.5 * torch.mean((discriminator(fake_data) - real_labels_ls) ** 2)

# Print outputs for Least Squares Loss
print("\nLeast Squares Loss:")
print("Discriminator output (real):", real_output)
print("Discriminator output (fake):", fake_output)
print("Discriminator loss (real):", loss_real_ls.item())
print("Discriminator loss (fake):", loss_fake_ls.item())
print("Total Discriminator Loss:", d_loss_ls.item())
print("Generator Loss:", g_loss_ls.item())


Binary Cross-Entropy Loss:
Discriminator output (real): tensor([[0.4859],
        [0.5076],
        [0.5850],
        [0.4937]], grad_fn=<SigmoidBackward0>)
Discriminator output (fake): tensor([[0.4885],
        [0.5176],
        [0.5045],
        [0.4825]], grad_fn=<SigmoidBackward0>)
Discriminator loss (real): 0.6604079604148865
Discriminator loss (fake): 0.690050482749939
Total Discriminator Loss: 1.3504583835601807
Generator Loss: 0.6970165967941284

Least Squares Loss:
Discriminator output (real): tensor([[0.4859],
        [0.5076],
        [0.5850],
        [0.4937]], grad_fn=<SigmoidBackward0>)
Discriminator output (fake): tensor([[0.4885],
        [0.5176],
        [0.5045],
        [0.4825]], grad_fn=<SigmoidBackward0>)
Discriminator loss (real): 0.11690548062324524
Discriminator loss (fake): 0.12422600388526917
Total Discriminator Loss: 0.2411314845085144
Generator Loss: 0.12596707046031952


KL (Kullback-Leibler) divergence loss is widely used in various contexts of deep neural networks (DNNs) and large language models (LLMs). Below are some key applications:

---

### **1. Knowledge Distillation**
- **Purpose**: To transfer knowledge from a larger model (teacher) to a smaller model (student).
- **Usage**: The student model is trained to minimize the KL divergence between the teacher's soft predictions (probability distribution over classes) and its own predictions.
- **Equation**:
  $$
  \text{KL}(P || Q) = \\sum P(x) \\log\\frac{P(x)}{Q(x)}
  $$
  where \(P(x)\) is the teacher's output distribution (softened with temperature), and \(Q(x)\) is the student's output.

---



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple teacher model
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc = nn.Linear(10, 5)  # Input size 10, output size 5

    def forward(self, x):
        return nn.functional.softmax(self.fc(x), dim=1)  # Softmax for probability distribution

# Define a simple student model
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc = nn.Linear(10, 5)  # Smaller capacity model with the same output size

    def forward(self, x):
        return nn.functional.softmax(self.fc(x), dim=1)  # Softmax for probability distribution

# Instantiate teacher and student models
teacher = TeacherModel()
student = StudentModel()

# Move models to the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
teacher.to(device)
student.to(device)

# Define a KL divergence loss
kl_loss = nn.KLDivLoss(reduction='batchmean')

# Create a dummy input tensor
input_data = torch.randn(16, 10).to(device)  # Batch size 16, input size 10

# Teacher model generates predictions
with torch.no_grad():  # Teacher is pre-trained, so no gradient updates are needed
    teacher_output = teacher(input_data)

# Define optimizer for the student model
optimizer = optim.Adam(student.parameters(), lr=0.01)

# Training loop for the student model
for epoch in range(10):  # Simple 10-epoch training
    student_output = student(input_data)  # Student's predictions

    # Compute KL divergence loss between teacher and student outputs
    loss = kl_loss(torch.log(student_output), teacher_output)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.4667
Epoch 2, Loss: 0.4288
Epoch 3, Loss: 0.3929
Epoch 4, Loss: 0.3590
Epoch 5, Loss: 0.3273
Epoch 6, Loss: 0.2976
Epoch 7, Loss: 0.2701
Epoch 8, Loss: 0.2445
Epoch 9, Loss: 0.2210
Epoch 10, Loss: 0.1995


### **2. Variational Autoencoders (VAEs)**
- **Purpose**: To learn a latent space representation of data.
- **Usage**: In VAEs, KL divergence regularizes the latent space by ensuring that the approximate posterior distribution \(q(z|x)\) is close to the prior distribution \(p(z)\), typically a standard normal distribution \(N(0, 1)\).
- **Loss Term**:
  $$
  \\text{KL}(q(z|x) || p(z)) = \\int q(z|x) \\log \\frac{q(z|x)}{p(z)} dz
  $$

---



In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the encoder network
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(10, 6)  # Input size 10, hidden size 6
        self.fc_mu = nn.Linear(6, 3)  # Latent mean
        self.fc_logvar = nn.Linear(6, 3)  # Latent log variance

    def forward(self, x):
        hidden = torch.relu(self.fc1(x))
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        return mu, logvar

# Define the decoder network
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(3, 6)  # Latent size 3, hidden size 6
        self.fc2 = nn.Linear(6, 10)  # Hidden size 6, output size 10

    def forward(self, z):
        hidden = torch.relu(self.fc1(z))
        reconstructed = torch.sigmoid(self.fc2(hidden))
        return reconstructed

# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        mu, logvar = self.encoder(x)
        # Reparameterization trick
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

# Instantiate the VAE model
vae = VAE()

# Move model to the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
vae.to(device)

# Define optimizer and reconstruction loss
optimizer = optim.Adam(vae.parameters(), lr=0.01)
reconstruction_loss_fn = nn.MSELoss(reduction='sum')  # Reconstruction loss

# Create a dummy input tensor
input_data = torch.randn(16, 10).to(device)  # Batch size 16, input size 10

# Training loop for the VAE
for epoch in range(10):  # Simple 10-epoch training
    reconstructed, mu, logvar = vae(input_data)  # Forward pass

    # Reconstruction loss
    reconstruction_loss = reconstruction_loss_fn(reconstructed, input_data)

    # KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Total loss
    total_loss = reconstruction_loss + kl_loss

    # Backpropagation and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print the losses for each epoch
    print(f"Epoch {epoch + 1}, Reconstruction Loss: {reconstruction_loss.item():.4f}, KL Loss: {kl_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")


Epoch 1, Reconstruction Loss: 196.6911, KL Loss: 3.5263, Total Loss: 200.2175
Epoch 2, Reconstruction Loss: 195.2261, KL Loss: 3.0601, Total Loss: 198.2862
Epoch 3, Reconstruction Loss: 194.2710, KL Loss: 2.6456, Total Loss: 196.9165
Epoch 4, Reconstruction Loss: 192.3088, KL Loss: 2.2894, Total Loss: 194.5983
Epoch 5, Reconstruction Loss: 191.4650, KL Loss: 1.9903, Total Loss: 193.4553
Epoch 6, Reconstruction Loss: 190.8412, KL Loss: 1.7417, Total Loss: 192.5830
Epoch 7, Reconstruction Loss: 190.4447, KL Loss: 1.5358, Total Loss: 191.9805
Epoch 8, Reconstruction Loss: 187.5156, KL Loss: 1.3771, Total Loss: 188.8927
Epoch 9, Reconstruction Loss: 187.4942, KL Loss: 1.2449, Total Loss: 188.7390
Epoch 10, Reconstruction Loss: 187.6893, KL Loss: 1.1573, Total Loss: 188.8466


### **3. Reinforcement Learning (Policy Optimization)**
- **Purpose**: To ensure stability in policy updates during training.
- **Usage**:
  - In **Trust Region Policy Optimization (TRPO)** or **Proximal Policy Optimization (PPO)**, KL divergence is used to limit the step size when updating the policy to prevent drastic changes.
  - KL divergence measures how different the new policy $$\\pi_{\\theta}(a|s)$$ is from the old policy $$\\pi_{\\text{old}}(a|s)$$.
- **Loss Term**:
  $$
  \\text{KL}(\\pi_{\\text{old}} || \\pi_{\\theta})
  $$

---


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Simple policy network for reinforcement learning
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 16)
        self.fc2 = nn.Linear(16, action_dim)

    def forward(self, x):
        return nn.functional.softmax(self.fc2(torch.relu(self.fc1(x))), dim=-1)

# Define a simple environment (e.g., 1D state space with discrete actions)
state_dim = 4  # State size
action_dim = 2  # Number of actions
policy_old = PolicyNetwork(state_dim, action_dim)
policy_new = PolicyNetwork(state_dim, action_dim)

# Move models to the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
policy_old.to(device)
policy_new.to(device)

# Define an optimizer for the new policy
optimizer = optim.Adam(policy_new.parameters(), lr=0.01)

# Dummy data for demonstration
batch_size = 16
states = torch.randn(batch_size, state_dim).to(device)  # Random states
old_probs = policy_old(states).detach()  # Probabilities from the old policy

# Training loop for policy updates with KL divergence
for epoch in range(10):  # Simple 10-epoch training
    # Get the new probabilities from the updated policy
    new_probs = policy_new(states)

    # Compute KL divergence between old and new policy probabilities
    kl_loss = torch.sum(old_probs * torch.log(old_probs / new_probs), dim=1).mean()

    # Add an optional entropy regularization term (to encourage exploration)
    entropy = -torch.sum(new_probs * torch.log(new_probs + 1e-10), dim=1).mean()
    total_loss = kl_loss - 0.01 * entropy  # Weight entropy with a small coefficient

    # Backpropagation and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print the losses for each epoch
    print(f"Epoch {epoch + 1}, KL Loss: {kl_loss.item():.4f}, Entropy: {entropy.item():.4f}, Total Loss: {total_loss.item():.4f}")

Epoch 1, KL Loss: 0.0087, Entropy: 0.6812, Total Loss: 0.0019
Epoch 2, KL Loss: 0.0053, Entropy: 0.6870, Total Loss: -0.0015
Epoch 3, KL Loss: 0.0041, Entropy: 0.6887, Total Loss: -0.0028
Epoch 4, KL Loss: 0.0032, Entropy: 0.6881, Total Loss: -0.0037
Epoch 5, KL Loss: 0.0023, Entropy: 0.6864, Total Loss: -0.0045
Epoch 6, KL Loss: 0.0017, Entropy: 0.6839, Total Loss: -0.0052
Epoch 7, KL Loss: 0.0015, Entropy: 0.6810, Total Loss: -0.0053
Epoch 8, KL Loss: 0.0017, Entropy: 0.6786, Total Loss: -0.0051
Epoch 9, KL Loss: 0.0017, Entropy: 0.6773, Total Loss: -0.0050
Epoch 10, KL Loss: 0.0016, Entropy: 0.6774, Total Loss: -0.0052


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define simple text and image encoders for contrastive learning
class TextEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super(TextEncoder, self).__init__()
        self.fc = nn.Linear(input_dim, embed_dim)

    def forward(self, x):
        return nn.functional.normalize(self.fc(x), p=2, dim=-1)  # L2 normalize

class ImageEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super(ImageEncoder, self).__init__()
        self.fc = nn.Linear(input_dim, embed_dim)

    def forward(self, x):
        return nn.functional.normalize(self.fc(x), p=2, dim=-1)  # L2 normalize

# Define dimensions
text_input_dim = 128  # Text embedding input size
image_input_dim = 256  # Image embedding input size
embed_dim = 64  # Common embedding dimension

# Instantiate encoders
text_encoder = TextEncoder(text_input_dim, embed_dim)
image_encoder = ImageEncoder(image_input_dim, embed_dim)

# Move models to the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
text_encoder.to(device)
image_encoder.to(device)

# Define an optimizer for both encoders
optimizer = optim.Adam(list(text_encoder.parameters()) + list(image_encoder.parameters()), lr=0.001)

# Dummy data for demonstration
batch_size = 16
text_embeddings = torch.randn(batch_size, text_input_dim).to(device)  # Simulated text embeddings
image_embeddings = torch.randn(batch_size, image_input_dim).to(device)  # Simulated image embeddings

# Training loop for contrastive learning
for epoch in range(10):  # Simple 10-epoch training
    # Encode text and image embeddings
    text_features = text_encoder(text_embeddings)
    image_features = image_encoder(image_embeddings)

    # Compute similarity matrices
    text_to_image_sim = torch.matmul(text_features, image_features.T)  # Text-to-image similarity
    image_to_text_sim = torch.matmul(image_features, text_features.T)  # Image-to-text similarity

    # Compute probability distributions
    text_probs = nn.functional.softmax(text_to_image_sim, dim=1)  # Row-wise softmax
    image_probs = nn.functional.softmax(image_to_text_sim, dim=1)  # Row-wise softmax

    # Ground truth: diagonal should be the highest (self-similarity)
    target_probs = torch.eye(batch_size).to(device)  # Identity matrix as ground truth

    # Compute KL divergence loss
    text_kl_loss = torch.sum(target_probs * torch.log(target_probs / text_probs), dim=1).mean()
    image_kl_loss = torch.sum(target_probs * torch.log(target_probs / image_probs), dim=1).mean()
    
    # Total loss (average of text-to-image and image-to-text losses)
    total_loss = (text_kl_loss + image_kl_loss) / 2

    # Backpropagation and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print the losses for each epoch
    print(f"Epoch {epoch + 1}, Text KL Loss: {text_kl_loss.item():.4f}, Image KL Loss: {image_kl_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")


Epoch 1, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 2, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 3, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 4, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 5, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 6, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 7, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 8, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 9, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan
Epoch 10, Text KL Loss: nan, Image KL Loss: nan, Total Loss: nan


# Focal Loss

### **Focal Loss: Usage**

Focal Loss is primarily designed to address the issue of class imbalance in classification tasks. It assigns more importance to hard-to-classify examples and reduces the weight for easily classified examples. Here's where it is commonly used:

---

### **1. Object Detection**
- Widely used in **single-stage object detectors** like RetinaNet.
- Helps the model focus on hard-to-detect objects (e.g., small, occluded, or overlapping objects) by reducing the loss contribution from well-classified examples (e.g., the background class in object detection).

---


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple object detection model (classification + bounding box regression)
class ObjectDetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(ObjectDetectionModel, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(16 * 16 * 16, num_classes)  # Classification head
        self.bbox_regressor = nn.Linear(16 * 16 * 16, 4)  # Bounding box regression head

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)  # Flatten features
        class_logits = self.classifier(features)
        bbox_deltas = self.bbox_regressor(features)
        return class_logits, bbox_deltas

# Define the Focal Loss for classification
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        """
        alpha: Balancing factor for class imbalance (e.g., positive vs negative classes).
        gamma: Focusing parameter that emphasizes hard-to-classify examples.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha # balances the importance of different classes, 
        # particularly in presense of class imbalance
        # α = 0.25 for positive class, and α = 0.75 for negative class
        # (for binary classification)., it reduces the contribution of the 
        # dominant class to the loss and increases focus on minority
        self.gamma = gamma # controls the focus on hard-to-classify images
        # for well classified examples (p_t closing on 1), (1 - p_t)^gamma
        # becomes small, reducing their contribution to total loss

    def forward(self, logits, targets):
        # Convert logits to probabilities using softmax
        probs = torch.softmax(logits, dim=-1)  # Shape: (batch_size, num_classes)

        # One-hot encode the target labels
        targets_one_hot = nn.functional.one_hot(targets, num_classes=logits.size(-1)).float()  # Shape: (batch_size, num_classes)

        # Extract probabilities for the true class (p_t)
        pt = torch.sum(probs * targets_one_hot, dim=-1)  # Shape: (batch_size,)

        # Compute the focal weight (alpha * (1 - p_t)^gamma)
        focal_weight = self.alpha * (1 - pt) ** self.gamma

        # Compute the focal loss (-focal_weight * log(p_t))
        loss = -focal_weight * torch.log(pt + 1e-8)  # Adding a small constant to avoid log(0)

        return loss.mean()  # Return the average loss

# Instantiate the model and loss functions
num_classes = 5
model = ObjectDetectionModel(num_classes=num_classes).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
classification_loss_fn = FocalLoss()
bbox_loss_fn = nn.SmoothL1Loss()  # Smooth L1 loss for bounding box regression

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data for demonstration
batch_size = 8
images = torch.randn(batch_size, 3, 32, 32).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Random images

# Random class labels for the batch
# Target classes are integers between 0 and num_classes-1
# Example: [2, 0, 1, 3, ...] for a batch size of 8
target_classes = torch.randint(0, num_classes, (batch_size,)).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Random bounding box coordinates for the batch
# Example: [[x1, y1, x2, y2], ...] for each image
# These are usually normalized to [0, 1] in real datasets
target_bboxes = torch.randn(batch_size, 4).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Training loop for object detection
for epoch in range(5):  # Simple 5-epoch training
    model.train()
    
    # Forward pass through the model
    class_logits, bbox_deltas = model(images)

    # Compute classification loss using Focal Loss
    # The loss emphasizes hard-to-classify examples using the focal weight
    cls_loss = classification_loss_fn(class_logits, target_classes)

    # Compute bounding box regression loss using Smooth L1 Loss
    # This loss penalizes large deviations between predicted and true bounding box coordinates
    bbox_loss = bbox_loss_fn(bbox_deltas, target_bboxes)

    # Total loss is the sum of classification and regression losses
    total_loss = cls_loss + bbox_loss

    # Backpropagation and optimization
    optimizer.zero_grad()  # Clear previous gradients
    total_loss.backward()  # Compute gradients
    optimizer.step()  # Update model parameters

    # Print the losses for each epoch
    print(f"Epoch {epoch + 1}, Classification Loss: {cls_loss.item():.4f}, BBox Loss: {bbox_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")


Epoch 1, Classification Loss: 0.2774, BBox Loss: 0.6539, Total Loss: 0.9313
Epoch 2, Classification Loss: 0.2704, BBox Loss: 0.7370, Total Loss: 1.0074
Epoch 3, Classification Loss: 0.1298, BBox Loss: 0.5171, Total Loss: 0.6468
Epoch 4, Classification Loss: 0.0565, BBox Loss: 0.3225, Total Loss: 0.3790
Epoch 5, Classification Loss: 0.0170, BBox Loss: 0.2213, Total Loss: 0.2383


### Explanation of Focal Loss Components

Focal Loss is a modification of the standard cross-entropy loss to address class imbalance in classification tasks by down-weighting the loss for well-classified examples and focusing on hard-to-classify ones. Here's how it works:

---

### **1. `γ` (Gamma)**
- **Purpose**: Controls the focus on hard-to-classify examples.
- **Mechanism**:
  - For well-classified examples (high probability for the true class, $$ p_t $$ near 1), the factor $$ (1 - p_t)^\gamma $$ becomes small, reducing their contribution to the total loss.
  - For hard-to-classify examples (low probability for the true class, $$ p_t $$ near 0), the factor $$ (1 - p_t)^\gamma $$ remains large, increasing their contribution.
- **Effect**: Higher $$ \gamma $$ increases the focus on harder examples. Common values are $$ \gamma = 2 $$ or $$ \gamma = 3 $$.

---

### **2. `α` (Alpha)**
- **Purpose**: Balances the importance of different classes, particularly in the presence of class imbalance.
- **Mechanism**:
  - If $$ \alpha $$ is set to 0.25 for the positive class and 0.75 for the negative class (for binary classification), it reduces the contribution of the dominant class (negative) to the loss and increases the focus on the minority class (positive).
- **Effect**: Helps balance the influence of underrepresented classes.

---

### **Focal Loss Formula**
For a single example:
$$
\text{FL}(p_t) = -\alpha (1 - p_t)^\gamma \log(p_t)
$$
Where:
- $$ p_t $$: Probability assigned to the true class.
- $$ \alpha $$: Balancing factor for class weights.
- $$ \gamma $$: Modulating factor for focusing on hard examples.

---

### **How it Works in the Code**
1. **Logits to Probabilities**:
   - `probs = torch.softmax(logits, dim=-1)` computes the probability distribution over classes.

2. **True Class Probabilities**:
   - `pt = torch.sum(probs * targets_one_hot, dim=-1)` extracts the predicted probabilities for the true class.

3. **Focal Weight**:
   - `focal_weight = self.alpha * (1 - pt) ** self.gamma` applies the modulating factor, emphasizing harder examples.

4. **Final Loss**:
   - `loss = -focal_weight * torch.log(pt + 1e-8)` computes the weighted log loss, where small $$ p_t $$ values contribute more due to the $$ (1 - p_t)^\gamma $$ term.

---

### Summary
- **`γ` (Gamma)**: Focuses on hard examples by down-weighting easy examples.
- **`α` (Alpha)**: Balances the contribution of different classes, addressing class imbalance.
- Combined, they make Focal Loss effective in scenarios like object detection with highly imbalanced classes (e.g., background vs. objects). 

Let me know if you'd like further clarification or an expanded explanation!


### **2. Imbalanced Datasets**
- Used in classification tasks with **severe class imbalance**, such as:
  - **Medical imaging**: Identifying rare diseases.
  - **Fraud detection**: Classifying rare fraudulent transactions.
  - **Anomaly detection**: Detecting rare events or behaviors.

---

### **3. Multi-label Classification**
- Applied in scenarios where multiple labels can be assigned to an input, and there is an imbalance in the distribution of labels (e.g., detecting attributes in an image).

---

### **4. Semantic Segmentation**
- Focuses on improving the classification of small or minority regions in segmentation tasks (e.g., segmenting rare object parts).

---

### **5. Natural Language Processing (NLP)**
- Useful in NLP tasks where certain classes are underrepresented, such as:
  - Named Entity Recognition (NER) for rare entities.
  - Sentiment analysis with imbalanced positive and negative reviews.

---

### **6. Multi-class Imbalance**
- Tasks like image classification with a **long-tailed distribution** of classes benefit from Focal Loss, as it reduces the dominance of frequent classes.

---

Focal Loss is particularly effective when you want to focus on difficult-to-classify examples and mitigate the effects of imbalanced datasets, making it a versatile choice in various machine learning domains. Let me know if you'd like examples or a detailed mathematical explanation!

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple object detection model (classification + bounding box regression)
class ObjectDetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(ObjectDetectionModel, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(16 * 16 * 16, num_classes)  # Classification head
        self.bbox_regressor = nn.Linear(16 * 16 * 16, 4)  # Bounding box regression head

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)  # Flatten features
        class_logits = self.classifier(features)
        bbox_deltas = self.bbox_regressor(features)
        return class_logits, bbox_deltas

# Define IoU loss for bounding box regression
class IoULoss(nn.Module):
    def __init__(self):
        super(IoULoss, self).__init__()

    def forward(self, pred_boxes, target_boxes):
        """
        pred_boxes: Predicted bounding boxes (batch_size, 4) [x1, y1, x2, y2]
        target_boxes: Ground truth bounding boxes (batch_size, 4) [x1, y1, x2, y2]
        """
        # Compute intersection
        x1 = torch.max(pred_boxes[:, 0], target_boxes[:, 0])
        y1 = torch.max(pred_boxes[:, 1], target_boxes[:, 1])
        x2 = torch.min(pred_boxes[:, 2], target_boxes[:, 2])
        y2 = torch.min(pred_boxes[:, 3], target_boxes[:, 3])

        intersection = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)

        # Compute areas of predicted and target boxes
        pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
        target_area = (target_boxes[:, 2] - target_boxes[:, 0]) * (target_boxes[:, 3] - target_boxes[:, 1])

        # Compute union
        union = pred_area + target_area - intersection

        # Compute IoU
        iou = intersection / (union + 1e-6)  # Add a small constant to avoid division by zero

        # IoU loss
        loss = 1 - iou.mean()  # Minimize 1 - IoU
        return loss

# Instantiate the model and loss functions
num_classes = 5
model = ObjectDetectionModel(num_classes=num_classes).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
classification_loss_fn = nn.CrossEntropyLoss()  # Cross-entropy for classification
bbox_loss_fn = IoULoss()  # IoU loss for bounding box regression

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data for demonstration
batch_size = 8
images = torch.randn(batch_size, 3, 32, 32).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Random images

# Random class labels for the batch
target_classes = torch.randint(0, num_classes, (batch_size,)).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Random class labels

# Random bounding box coordinates for the batch
target_bboxes = torch.randn(batch_size, 4).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Random ground truth boxes
pred_bboxes = torch.randn(batch_size, 4).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Predicted boxes

# Training loop for object detection
for epoch in range(5):  # Simple 5-epoch training
    model.train()

    # Forward pass through the model
    class_logits, bbox_deltas = model(images)

    # Compute classification loss
    cls_loss = classification_loss_fn(class_logits, target_classes)

    # Compute IoU loss for bounding boxes
    bbox_loss = bbox_loss_fn(bbox_deltas, target_bboxes)

    # Total loss is the sum of classification and IoU losses
    total_loss = cls_loss + bbox_loss

    # Backpropagation and optimization
    optimizer.zero_grad()  # Clear previous gradients
    total_loss.backward()  # Compute gradients
    optimizer.step()  # Update model parameters

    # Print the losses for each epoch
    print(f"Epoch {epoch + 1}, Classification Loss: {cls_loss.item():.4f}, IoU Loss: {bbox_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")

Epoch 1, Classification Loss: 1.6428, IoU Loss: 1.0000, Total Loss: 2.6428
Epoch 2, Classification Loss: 1.0660, IoU Loss: 1.0000, Total Loss: 2.0660
Epoch 3, Classification Loss: 0.6803, IoU Loss: 1.0000, Total Loss: 1.6803
Epoch 4, Classification Loss: 0.3360, IoU Loss: 1.0000, Total Loss: 1.3360
Epoch 5, Classification Loss: 0.1691, IoU Loss: 1.0000, Total Loss: 1.1691


# DICE Loss

### **Dice Loss: Overview**

**Dice Loss** is a loss function designed for **binary and multi-class segmentation tasks**. It measures the overlap between the predicted segmentation and the ground truth, focusing on minimizing differences in areas of the objects being segmented.

---

### **Mathematical Definition**
The **Dice Coefficient** (or Dice Similarity Coefficient, DSC) measures the similarity between two sets (predicted and ground truth masks):
$$
\text{Dice Coefficient} = \frac{2 |A \cap B|}{|A| + |B|}
$$
Where:
- \(A\) = Predicted mask.
- \(B\) = Ground truth mask.

The **Dice Loss** is defined as:
$$
\text{Dice Loss} = 1 - \text{Dice Coefficient}
$$

---

### **Applications**
1. **Image Segmentation**:
   - Used in tasks such as medical imaging (e.g., tumor segmentation in MRI scans).
   - Ensures better overlap between predicted and actual segmentations.

2. **Imbalanced Data**:
   - Particularly effective when dealing with highly imbalanced classes (e.g., small objects in large images).
   - Penalizes false positives and false negatives equally.

3. **Binary and Multi-Class Segmentation**:
   - Can be extended for multi-class segmentation tasks by averaging Dice Loss across all classes.

---

### **Advantages**
1. Focuses on overlap, which is more relevant for segmentation tasks than simple pixel-wise losses (like cross-entropy).
2. Handles class imbalance better, as it normalizes by the total size of the masks.


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Dice Loss
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, preds, targets):
        """
        preds: Predicted segmentation map (batch_size, num_classes, height, width)
        targets: Ground truth segmentation map (batch_size, num_classes, height, width)
        """
        smooth = 1.0  # To avoid division by zero
        preds = preds.softmax(dim=1)  # Apply softmax to get probabilities

        # Flatten predictions and targets for Dice computation
        preds_flat = preds.view(preds.size(0), preds.size(1), -1)  # (batch_size, num_classes, -1)
        targets_flat = targets.view(targets.size(0), targets.size(1), -1)  # (batch_size, num_classes, -1)

        # Compute intersection and union
        intersection = (preds_flat * targets_flat).sum(dim=2)  # Intersection per class
        union = preds_flat.sum(dim=2) + targets_flat.sum(dim=2)  # Union per class

        # Dice coefficient
        dice = (2.0 * intersection + smooth) / (union + smooth)

        # Average over batch and classes
        dice_loss = 1 - dice.mean()
        return dice_loss

# Dummy data for demonstration
batch_size = 4
num_classes = 3
height, width = 64, 64

# Predictions (logits) from a model
preds = torch.randn(batch_size, num_classes, height, width).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Ground truth segmentation maps (one-hot encoded)
targets = torch.randint(0, 2, (batch_size, num_classes, height, width)).float().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Instantiate Dice Loss
dice_loss_fn = DiceLoss()

# Compute Dice Loss
loss = dice_loss_fn(preds, targets)
print(f"Dice Loss: {loss.item():.4f}")

Dice Loss: 0.6024


### **Perceptual Loss: Where It Is Used**

Perceptual loss measures the perceptual similarity between images rather than pixel-wise differences, focusing on how humans perceive visual differences. It relies on pre-trained deep neural networks, such as VGG, to compare features from intermediate layers. Here are its primary use cases:

---

### **1. Image Super-Resolution**
- Perceptual loss is used to train models that generate high-resolution images from low-resolution inputs.
- It ensures the output image is visually similar to the ground truth in terms of texture and structure, beyond pixel-level accuracy.

---

### **2. Image-to-Image Translation**
- Applied in tasks like style transfer, domain adaptation, and image colorization.
- Perceptual loss ensures that the transformed image retains the content structure of the input while matching the desired target style.

---

### **3. Generative Adversarial Networks (GANs)**
- In tasks like image synthesis and inpainting, perceptual loss helps improve the quality of the generated image by aligning it with human perception.

---

### **4. Neural Style Transfer**
- Used to minimize differences in content and style between the input and target images.
- Content loss focuses on structural similarity, while style loss ensures stylistic consistency.

---

### **5. Video Frame Prediction**
- Ensures temporal consistency in videos by preserving perceptual quality across consecutive frames.

---

### **6. 3D Reconstruction**
- Helps in aligning 3D models or reconstructed objects with 2D image views, improving texture and geometry quality.

---

### **7. Image Denoising and Deblurring**
- Used to improve perceptual quality by focusing on restoring image details as seen by humans.

---

### **8. Face Reconstruction and Alignment**
- Ensures facial landmarks and features are preserved during tasks like face swapping, alignment, or super-resolution.

---

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple perceptual loss using a pre-trained VGG-like network
class PerceptualLoss(nn.Module):
    def __init__(self, feature_extractor):
        super(PerceptualLoss, self).__init__()
        self.feature_extractor = feature_extractor  # Pre-trained feature extractor
        for param in self.feature_extractor.parameters():
            param.requires_grad = False  # Freeze the feature extractor

    def forward(self, input_image, target_image):
        """
        input_image: Generated image (batch_size, channels, height, width)
        target_image: Ground truth image (batch_size, channels, height, width)
        """
        input_features = self.feature_extractor(input_image)
        target_features = self.feature_extractor(target_image)
        loss = nn.functional.mse_loss(input_features, target_features)  # MSE on feature maps
        return loss

# Define a simple feature extractor (e.g., part of a VGG-like network)
class SimpleFeatureExtractor(nn.Module):
    def __init__(self):
        super(SimpleFeatureExtractor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.features(x)

# Instantiate the feature extractor and perceptual loss
feature_extractor = SimpleFeatureExtractor().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
perceptual_loss_fn = PerceptualLoss(feature_extractor)

# Dummy data for demonstration
batch_size = 4
input_images = torch.randn(batch_size, 3, 64, 64).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Generated images
target_images = torch.randn(batch_size, 3, 64, 64).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Ground truth images

# Compute perceptual loss
loss = perceptual_loss_fn(input_images, target_images)
print(f"Perceptual Loss: {loss.item():.4f}")


Perceptual Loss: 0.0199


# CTC Loss

### **CTC Loss: Overview**

**Connectionist Temporal Classification (CTC) Loss** is designed for sequence prediction tasks where:
1. The alignment between input and output sequences is unknown.
2. The input and output sequence lengths may differ.

It is commonly used in applications where outputs have varying lengths, such as speech recognition, handwriting recognition, and OCR (optical character recognition).

---

### **Key Features**
1. **Alignment-Free**: 
   - CTC automatically handles the alignment of input and output sequences.
   - For example, in speech recognition, the model predicts a probability distribution over characters for each time step of the audio, and CTC determines the best alignment.

2. **Blank Tokens**: 
   - CTC introduces a special "blank" token to handle varying sequence lengths.
   - Blank tokens are used to fill gaps between output tokens when no explicit output is required.

3. **Collapse Repeated Tokens**: 
   - CTC maps repeated predictions (e.g., "hhheeelllooo") and blanks to the target sequence ("hello").

---

### **Applications**
1. **Speech Recognition**:
   - Used in end-to-end speech models like DeepSpeech.
   - Converts acoustic feature sequences into text without requiring pre-aligned transcriptions.
   
2. **Handwriting Recognition**:
   - Maps handwritten strokes to characters or words.

3. **Optical Character Recognition (OCR)**:
   - Used in systems where text is extracted from images without strict alignment between input pixels and output characters.

4. **Sign Language Recognition**:
   - Predicts the sequence of words or letters corresponding to gestures in sign language.

---

### **How It Works**
1. The model outputs a probability distribution over the set of possible characters (including the blank token) for each time step.
2. CTC computes the total probability of all valid alignments that map to the target sequence.
3. The loss minimizes the negative log probability of the correct output sequence given the input sequence.

---

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple Transformer-based model for sequence prediction
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_classes, num_heads=2, num_layers=2):
        super(SimpleTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(model_dim, num_classes + 1)  # +1 for the blank token

    def forward(self, x):
        x = self.embedding(x)  # Project input to model dimension
        x = self.transformer(x)  # Pass through transformer layers
        x = self.fc(x)  # Project to class probabilities (including blank)
        return x

# Instantiate the model
input_dim = 13  # Example input dimension (e.g., MFCCs for speech)
model_dim = 64  # Model dimension
num_classes = 10  # Number of output classes

model = SimpleTransformer(input_dim, model_dim, num_classes).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define the CTC Loss
ctc_loss_fn = nn.CTCLoss(blank=num_classes, reduction='mean', zero_infinity=True)

# Dummy data for demonstration
batch_size = 4
sequence_length = 15  # Length of input sequences
output_length = 5  # Length of target sequences

# Inputs (e.g., MFCC features for speech)
inputs = torch.randn(sequence_length, batch_size, input_dim).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Targets (e.g., token indices)
targets = torch.randint(0, num_classes, (batch_size * output_length,)).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Input and target lengths
input_lengths = torch.full((batch_size,), sequence_length, dtype=torch.long).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
target_lengths = torch.full((batch_size,), output_length, dtype=torch.long).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Forward pass
logits = model(inputs)  # Shape: (sequence_length, batch_size, num_classes + 1)
log_probs = nn.functional.log_softmax(logits, dim=-1)  # Log probabilities for CTC

# Compute CTC Loss
loss = ctc_loss_fn(log_probs, targets, input_lengths, target_lengths)
print(f"CTC Loss: {loss.item():.4f}")


CTC Loss: 5.3860


# Triplet loss

### **Triplet Loss: Overview**

**Triplet Loss** is a type of loss function used for **metric learning**. It ensures that embeddings of similar items are closer together while embeddings of dissimilar items are farther apart in the feature space.

---

### **How It Works**
Triplet loss operates on **triplets of data**:
1. **Anchor**: The reference data point.
2. **Positive**: A data point similar to the anchor.
3. **Negative**: A data point dissimilar to the anchor.

The loss is designed to minimize the distance between the **anchor** and **positive** embeddings while maximizing the distance between the **anchor** and **negative** embeddings.

The mathematical formulation:
\[
\mathcal{L}_{\text{triplet}} = \max \left( d(a, p) - d(a, n) + \text{margin}, 0 \\right)
\]
Where:
- \( a, p, n \): Anchor, Positive, and Negative embeddings.
- \( d(x, y) \): Distance metric (usually L2 or cosine distance).
- **Margin**: A predefined constant that defines the minimum required separation between \( d(a, p) \) and \( d(a, n) \).

---

### **Applications**
1. **Face Recognition**:
   - Used in models like FaceNet to ensure embeddings of the same person are closer than embeddings of different people.
2. **Image Retrieval**:
   - Ensures that visually similar images are closer in the embedding space.
3. **Speaker Verification**:
   - Verifies if two audio samples belong to the same speaker.
4. **Signature Verification**:
   - Determines whether two signatures belong to the same person.
5. **Product Recommendation**:
   - Ensures embeddings of similar products are closer together in recommendation systems.

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple embedding model for triplet loss
class SimpleEmbeddingModel(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(SimpleEmbeddingModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, embedding_dim)  # Project to embedding space
        )

    def forward(self, x):
        return nn.functional.normalize(self.fc(x), p=2, dim=-1)  # Normalize embeddings

# Instantiate the model
input_dim = 10  # Input feature dimension
embedding_dim = 5  # Embedding space dimension
model = SimpleEmbeddingModel(input_dim, embedding_dim).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define Triplet Loss
triplet_loss_fn = nn.TripletMarginLoss(margin=1.0, p=2)  # L2 distance with margin 1.0

# Dummy data for demonstration
batch_size = 4
anchor = torch.randn(batch_size, input_dim).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Anchor samples
positive = torch.randn(batch_size, input_dim).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Positive samples
negative = torch.randn(batch_size, input_dim).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Negative samples

# Forward pass to compute embeddings
anchor_embed = model(anchor)
positive_embed = model(positive)
negative_embed = model(negative)

# Compute Triplet Loss
loss = triplet_loss_fn(anchor_embed, positive_embed, negative_embed)
print(f"Triplet Loss: {loss.item():.4f}")


---

### **1. Docker Basics**
- `docker --version` – Check Docker version.
- `docker --help` – Get help for Docker commands.
- `docker <subcommand> --help` – Get help for a specific subcommand.

---

### **2. Working with Containers**
- `docker run <image>` – Run a container.
- `docker ps` – List running containers.
- `docker ps -a` – List all containers (including stopped ones).
- `docker start <container_id>` – Start a stopped container.
- `docker stop <container_id>` – Stop a running container.
- `docker restart <container_id>` – Restart a container.
- `docker rm <container_id>` – Remove a container.
- `docker logs <container_id>` – View logs of a container.
- `docker exec -it <container_id> /bin/bash` – Access a container’s shell.
- `docker kill <container_id>` – Force-stop a container.

---

### **3. Working with Images**
- `docker images` – List all available images.
- `docker pull <image>` – Download an image from a registry.
- `docker build -t <image_name>:<tag> <path>` – Build an image from a `Dockerfile`.
- `docker tag <source_image> <target_image>` – Tag an image with a new name.
- `docker rmi <image_id>` – Remove an image.

---

### **4. Docker Compose**
- `docker-compose up` – Start all services in the `docker-compose.yml`.
- `docker-compose up --build` – Rebuild images and start services.
- `docker-compose down` – Stop and remove services and containers.
- `docker-compose ps` – List services managed by Docker Compose.
- `docker-compose logs` – View logs for services.

---

### **5. Docker Networks**
- `docker network ls` – List all networks.
- `docker network create <network_name>` – Create a new network.
- `docker network inspect <network_name>` – View details of a network.
- `docker network rm <network_name>` – Remove a network.

---

### **6. Docker Volumes**
- `docker volume ls` – List all volumes.
- `docker volume create <volume_name>` – Create a new volume.
- `docker volume inspect <volume_name>` – Inspect a volume.
- `docker volume rm <volume_name>` – Remove a volume.

---

### **7. System Cleanup**
- `docker system prune` – Remove unused containers, images, networks, and volumes.
- `docker container prune` – Remove stopped containers.
- `docker image prune` – Remove unused images.
- `docker volume prune` – Remove unused volumes.

---

### **8. Inspecting and Debugging**
- `docker inspect <container_id>` – View details of a container.
- `docker stats` – Monitor resource usage of running containers.
- `docker top <container_id>` – Display running processes inside a container.

---
